In [1]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

In [12]:
import random

def bs_price(S,K,sigma,r,T, year_rate = 252, option_type = "call"):
    '''
    bs 期权定价 输出为期权价格
    输入格式：
    S:标的价格
    K:行权价
    sigma:波动率
    r:无风险利率
    T:年华剩余期限
    year_rate:年化系数
    '''

    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option_type == 'call':
        # return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return np.maximum(S - K, 0) if T==0 else S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else :
        # return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
        return np.maximum(K-S,0) if T==0 else K*np.exp(-r*T)*norm.cdf(-1*d2) - S*norm.cdf(-1*d1)

def bs_vega(S,K,sigma,r,T ,year_rate = 252):
    '''
    计算vega
    '''
    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(np.abs(T)) )
    return S*norm.pdf(d1)*np.sqrt(np.abs(T))
    # return S * np.sqrt(abs(T)) * np.exp(-pow(d1,2)/2) / np.sqrt(2*np.pi)

def iv_bs_bisection(S, K, r, T, price, option_type, iv_uplimit = 0.985, iv_downlimit = 0.015, precision = 3, year_rate = 365, max_iterations = 400):
    '''二分法求iv'''
    left_iv, right_iv = iv_downlimit, iv_uplimit
    # left_price = BS_price(S,K,left_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # right_price= BS_price(S,K,right_iv, r, T,  year_rate = year_rate, option_type = option_type)
    mid_iv = (left_iv + right_iv)/2
    mid_price = bs_price(S,K,mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # print(f"first, the mid price is {mid_price}, the price is {price}")
    cnt = 0
    while abs(price - mid_price) >= 0.1**precision and cnt < max_iterations:
        if mid_price < price :
            left_iv = mid_iv
        else:
            right_iv = mid_iv
        mid_iv = (left_iv + right_iv)/2
        # if mid_iv < iv_uplimit or mid_iv > iv_downlimit:
        #     break
        # print(f"cnt = {cnt}, the left is{left_iv}, the right is{right_iv}, the mid is {mid_iv}")
        cnt += 1
        mid_price = bs_price(S, K, mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    return mid_iv

def find_vol_newton( S, K, r, T, target_value, option_type, start_sigma = 0.5, precision = 3, year_rate = 252, max_iterations = 200):
    '''迭代法求iv'''
    sigma = start_sigma
    for i in range(0, max_iterations):
        bs_price_ = bs_price(S,K,sigma,r,T, year_rate = year_rate, option_type = option_type)
        vega = bs_vega(S, K, T, r, sigma, year_rate = year_rate)*100
        diff = target_value - bs_price_  # our root
        if (abs(diff) < 0.1**precision):
            return sigma
        sigma = sigma + diff/(vega) # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

def change(type_name):
    if type_name == '认购':
        return "call"
    elif type_name == '认沽':
        return "put"
    else:
        return "wrong_type"

In [13]:
#数据处理
alldata = []
rate = pd.read_excel(r".\data\unrisked_rate.xlsx")
rate['date'] = pd.to_datetime(rate['date'])
rate['Value'] = rate['Value']/100

etf50 = pd.read_excel(r'.\data\50etf_df.xlsx')
etf50 = etf50[['Date','close']]
etf50['Date'] = pd.to_datetime(etf50['Date'])

csv_folder = r'.\data\50etf_option_data_csv'
output_folder = r'.\data\out'
csv_list = os.listdir(csv_folder)

# ll = 1139
for csv_name in csv_list:
    print(csv_name)
    '''ll -= 1
    if ll > 0:
        continue'''
    perdata = []
    csv_path = csv_folder +"\\"+csv_name
    output_path = output_folder + "\\" + csv_name
    data = pd.read_csv(csv_path, encoding = 'GBK', index_col= False)

    data = data[['date','ptmtradeday','exe_price', 'open', 'high','low','close', 'call_or_put']]
    data["date"] = pd.to_datetime(data["date"])
    data['call_or_put'] = data['call_or_put'].apply(change)
    if data['open'].dtype == 'object':
        data.drop(data[data.open == '非数字'].index, inplace=True)

    merged_option1 = pd.merge(rate, data, on = 'date', how = 'inner')
    merged_option2 = pd.merge(etf50, merged_option1, left_on = 'Date', right_on = 'date', how = 'inner')
    merged_option2 = merged_option2[['call_or_put','date','ptmtradeday','exe_price','Value', 'open', 'high','low','close_y','close_x']]
    merged_option2.columns = ['call_or_put', 'date','ptmtradeday','k', 'rate', 'open', 'high','low','close','s']

    for i in range(merged_option2.shape[0]):
        merged_option2['close'][i] = float(merged_option2['close'][i])
        tmp = iv_bs_bisection(np.array(merged_option2['s'][i]), np.array(merged_option2['k'][i]), np.array(merged_option2['rate'][i]), np.array(merged_option2['ptmtradeday'][i]), np.array(merged_option2['close'][i]), np.array(merged_option2['call_or_put'][i]))
        if abs(tmp - 0.985) < 0.005 or abs(tmp - 0.015) < 0.005 or abs(tmp - 0.5) < 0.005:
            if i == 0:
                tmp = tmp
            else:
                tmp = perdata[i-1] + random.random()/100 - 0.005

        perdata.append(tmp)

    merged_option2['iv'] = perdata
    merged_option2.to_csv(output_path)


10000001.csv
10000002.csv
10000003.csv
10000004.csv
10000005.csv
10000006.csv
10000007.csv
10000008.csv
10000009.csv
10000010.csv
10000011.csv
10000012.csv
10000013.csv
10000014.csv
10000015.csv
10000016.csv
10000017.csv
10000018.csv
10000019.csv
10000020.csv
10000021.csv
10000022.csv
10000023.csv
10000024.csv
10000025.csv
10000026.csv
10000027.csv
10000028.csv
10000029.csv
10000030.csv
10000031.csv
10000032.csv
10000033.csv
10000034.csv
10000035.csv
10000036.csv
10000037.csv
10000038.csv
10000039.csv
10000040.csv
10000041.csv
10000042.csv
10000043.csv
10000044.csv
10000045.csv
10000046.csv
10000047.csv
10000048.csv
10000049.csv
10000050.csv
10000051.csv
10000052.csv
10000053.csv
10000054.csv
10000055.csv
10000056.csv
10000057.csv
10000058.csv
10000059.csv
10000060.csv
10000061.csv
10000062.csv
10000063.csv
10000064.csv
10000065.csv
10000066.csv
10000067.csv
10000068.csv
10000069.csv
10000070.csv
10000071.csv
10000072.csv
10000073.csv
10000074.csv
10000075.csv
10000076.csv
10000077.csv

KeyboardInterrupt: 

In [2]:
#拼训练数据
csv_folder = r'.\data\out'
output_folder = r'.\data'
csv_list = os.listdir(csv_folder)
output_path = output_folder + "\\" + 'final_data.csv'

call_X = []
call_Y = []
put_X = []
put_Y = []
# 加一个混合
# 做一个对比

#numbers = 1139

for csv_name in csv_list:
    if int(csv_name.split('.')[0]) % 100 == 0:
        print(csv_name)
    perdata = []
    '''numbers -= 1
    if numbers == 0:
        break'''
    csv_path = csv_folder +"\\"+csv_name
    data = pd.read_csv(csv_path, encoding = 'GBK', index_col= False)
    if len(data) < 11:
        continue

    datatype = data['call_or_put'][0]
    ll = 0

    for i in range(len(data)-1):
        onedata = list(data.iloc[i,3:])
        perdata.append(onedata)
        ll += 1
        if ll == 10:
            if datatype == 'call':
                call_X.append(perdata)
                call_Y.append(data['iv'][i+1])
            else:
                put_X.append(perdata)
                put_Y.append(data['iv'][i+1])
            ll -= 1
            perdata = perdata[1:]


10000100.csv
10000200.csv
10000300.csv
10000400.csv
10000500.csv
10000600.csv
10000700.csv
10000800.csv
10000900.csv
10001000.csv
10001100.csv
10001200.csv
10001300.csv
10001400.csv
10001500.csv
10001600.csv
10001700.csv
10001800.csv
10001900.csv
10002000.csv
10002100.csv
10002200.csv


In [10]:
teststring = '100100.csv'
print(teststring.split('.')[0])
print(int(teststring.split('.')[0]) % 100)

100100
0


In [8]:
call_X = np.array(call_X)
call_Y = np.array(call_Y)

put_X = np.array(put_X)
put_Y = np.array(put_Y)

# 数据处理
for i in range(len(call_X)):
    if call_Y[i] <= 0:
        call_Y[i] = -call_Y[i]
        print(i, call_Y[i])
    for j in range(10):
        for k in range(1, 9):
            if call_X[i][j][k] <= 0:
                call_X[i][j][k] = -call_X[i][j][k]
                print(i,j,k,call_X[i][j][k])

for i in range(len(put_X)):
    if put_Y[i] <= 0:
        put_Y[i] = -put_Y[i]
        print(i, put_Y[i])
    for j in range(10):
        for k in range(1, 9):
            if put_X[i][j][k] <= 0:
                put_X[i][j][k] = -put_X[i][j][k]
                print(i,j,k,put_X[i][j][k])
# 正则化


12075 0.00286393148233
12076 3.59737517164e-05
12077 0.00297874411765
12078 0.000318559103993
12092 0.00410451302802
12093 0.00225836145527
12095 0.00184864933953
26088 0.00244463288221
26089 0.00411922363615
26090 0.00630291729092
26091 0.0105671176137
26092 0.0112089789661
26093 0.00858699003753
26094 0.00398529588666
26096 0.00455974837495
26097 0.00887373301837
26098 0.0114674415614
26099 0.0094961026026
26100 0.00860402921986
26101 0.00463112669689
26102 0.00416382105174
26103 0.00505357421686
26104 0.010027647453
26105 0.00749296630298
26106 0.00698461148705
26107 0.00526612688848
26108 0.00966071213234
26109 0.0109322635728
26110 0.00801595809705
26111 0.00333193738274
26112 0.00212004144553
26113 0.00591131100696
26114 0.00467923608703
26115 0.00212724501505
26117 0.00153714700592
26118 0.00277143473715
26866 0.00555057445115
26867 0.00880277866473
26868 0.00617870925902
26869 0.00717230586553
26870 0.0112993984634
26871 0.0145154916113
26872 0.0111374406934
26873 0.01037115716

In [9]:
from sklearn.model_selection import train_test_split

call_train_X, call_val_X, call_train_Y, call_val_Y = train_test_split(call_X,
                                                                      call_Y,
                                                                      test_size=0.2,
                                                                      random_state=19011)

put_train_X, put_val_X, put_train_Y, put_val_Y = train_test_split(put_X,
                                                                  put_Y,
                                                                  test_size=0.2,
                                                                  random_state=19011)

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf

from matplotlib import pyplot as plt


def Cov_model():
    model = Sequential()
    model.add(Conv1D(32, 2, padding='valid', input_shape=(10, 9), activation="selu"))
    model.add(Conv1D(64, 2, padding='valid', activation="selu"))
    model.add(Conv1D(128, 2, padding='valid', activation="selu"))
    model.add(Conv1D(256, 2, padding='valid', activation="selu"))
    #model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    opt = Adam(lr=0.001, clipnorm=1)
    model.compile(optimizer=opt, loss=tf.keras.losses.mean_squared_error,
                  metrics=['mae'])
    model.summary()
    return model

# fit network
cov_model = Cov_model()

history = cov_model.fit(call_train_X[10000:15000], call_train_Y[10000:15000],
                    epochs=50, batch_size=256,
                    validation_data=(call_val_X[100:150], call_val_Y[100:150]),
                    verbose=2,
                    shuffle=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_37 (Conv1D)           (None, 9, 32)             608       
_________________________________________________________________
conv1d_38 (Conv1D)           (None, 8, 64)             4160      
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 7, 128)            16512     
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 6, 256)            65792     
_________________________________________________________________
flatten_10 (Flatten)         (None, 1536)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 1537      
Total params: 88,609
Trainable params: 88,609
Non-trainable params: 0
_________________________________________________________________
Train 

KeyboardInterrupt: 

In [68]:
print(call_train_X[0][0])

54092

12076 9 8 0.00286393148233
12077 8 8 0.00286393148233
12077 9 8 3.59737517164e-05
12078 7 8 0.00286393148233
12078 8 8 3.59737517164e-05
12078 9 8 0.00297874411765
12079 6 8 0.00286393148233
12079 7 8 3.59737517164e-05
12079 8 8 0.00297874411765
12079 9 8 0.000318559103993
12080 5 8 0.00286393148233
12080 6 8 3.59737517164e-05
12080 7 8 0.00297874411765
12080 8 8 0.000318559103993
12081 4 8 0.00286393148233
12081 5 8 3.59737517164e-05
12081 6 8 0.00297874411765
12081 7 8 0.000318559103993
12082 3 8 0.00286393148233
12082 4 8 3.59737517164e-05
12082 5 8 0.00297874411765
12082 6 8 0.000318559103993
12083 2 8 0.00286393148233
12083 3 8 3.59737517164e-05
12083 4 8 0.00297874411765
12083 5 8 0.000318559103993
12084 1 8 0.00286393148233
12084 2 8 3.59737517164e-05
12084 3 8 0.00297874411765
12084 4 8 0.000318559103993
12085 0 8 0.00286393148233
12085 1 8 3.59737517164e-05
12085 2 8 0.00297874411765
12085 3 8 0.000318559103993
12086 0 8 3.59737517164e-05
12086 1 8 0.00297874411765
12086 2 8 0

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
'''画图'''
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import pyecharts.options as opts
from pyecharts.charts import Line

output_folder = r'.\data\out'
csv_list = os.listdir(output_folder)

for datafile in csv_list:
    csv_path = output_folder +"\\"+ datafile
    data = pd.read_csv(csv_path, encoding = 'utf-8', index_col= False)
    data = data['iv']
    x = list(range(1,len(data)+1))
    #print('x',x)
    #print('data', data)
    plt.plot(x, data, 'ro-', color='#4169E1', alpha=0.8, linewidth=1, label='iv折线图')
    plt.savefig('./data/out_pic/'+ datafile + '.jpg')
    plt.clf()
    line = (
        Line()
        .set_global_opts(
            tooltip_opts=opts.TooltipOpts(is_show=False),
            xaxis_opts=opts.AxisOpts(type_="value"),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
        )
        .add_xaxis(xaxis_data=x)
        .add_yaxis(
            series_name="基本折线图",
            y_axis=data,
            symbol="emptyCircle",
            is_symbol_show=True,
            label_opts=opts.LabelOpts(is_show=False),
        )
    )
    line.render('./data/out_pic/'+ datafile + '.html')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, Flatten


# design network
def model():
    model = Sequential()
    model.add(Conv1D(32, 2, padding='valid', input_shape=(10,10), activation="relu"))
    model.add(Conv1D(62, 2, padding='valid', activation="relu"))
    model.add(Conv1D(128, 2, padding='valid', activation="relu"))
    model.add(Conv1D(256, 2, padding='valid', activation="relu"))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    # plot history
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()

    return model

In [ ]:
def model1():
    model = Sequential()
    model.add(LSTM(10, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    # plot history
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()

    return model1
